In [ ]:
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from sklearn.decomposition import PCA

In [ ]:

def pansharpening_pca(pan_data, pan_profile, ms_data, ms_profile, output_path):
    """
    Realiza o pansharpening de uma imagem multiespectral (MS) usando uma imagem pancromática (PAN) através do método PCA.

    Args:
    - pan_data: Dados da imagem pancromática.
    - pan_profile: Metadados da imagem pancromática.
    - ms_data: Dados da imagem multiespectral.
    - ms_profile: Metadados da imagem multiespectral.
    - output_path: Caminho para salvar a imagem resultante do pansharpening.

    """
    # Reamostrar a imagem multiespectral para corresponder à resolução da imagem pancromática usando vizinho mais próximo
    ms_data_resampled = np.empty((ms_data.shape[0], pan_data.shape[0], pan_data.shape[1]), dtype=ms_data.dtype)
    for i in range(ms_data.shape[0]):
        reproject(
            source=ms_data[i],
            destination=ms_data_resampled[i],
            src_transform=ms_profile['transform'],
            src_crs=ms_profile['crs'],
            dst_transform=pan_profile['transform'],
            dst_crs=pan_profile['crs'],
            resampling=Resampling.nearest
        )
    
    # Reorganizar dados para PCA
    reshaped_ms = ms_data_resampled.reshape(ms_data_resampled.shape[0], -1).T  # Transpor para ter pixels como linhas
    pca = PCA(n_components=ms_data_resampled.shape[0])
    pca.fit(reshaped_ms)
    ms_pca = pca.transform(reshaped_ms)
    
    # Substituir a primeira componente principal pela imagem pancromática
    pan_flat = pan_data.flatten()
    ms_pca[:, 0] = pan_flat
    
    # Reconstruir imagem multiespectral do PCA
    ms_reconstructed = pca.inverse_transform(ms_pca).T.reshape(ms_data_resampled.shape)
    
    # Salvar a imagem resultante
    output_profile = ms_profile.copy()
    output_profile.update({
        'height': pan_data.shape[0],
        'width': pan_data.shape[1],
        'transform': pan_profile['transform']
    })
    
    with rasterio.open(output_path, 'w', **output_profile) as dest:
        dest.write(ms_reconstructed)


In [ ]:
# pan_path = 
# ms_path = 


with rasterio.open('caminho_para_imagem_pancromatica.tif') as pan_src:
     pan_data = pan_src.read(1)
     pan_profile = pan_src.profile

with rasterio.open('caminho_para_imagem_multiespectral.tif') as ms_src:
    ms_data = ms_src.read()
    ms_profile = ms_src.profile


pansharpening_pca(pan_data, pan_profile, ms_data, ms_profile, 'caminho_para_salvar_imagem_resultante.tif')